In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('lahman.sqlite')

In [3]:
query="""
SELECT * FROM sqlite_master;
"""

In [4]:
df_schema = pd.read_sql_query(query, conn)
df_schema.tbl_name.unique()

array(['allstarfull', 'appearances', 'awardsmanagers', 'awardsplayers',
       'awardssharemanagers', 'awardsshareplayers', 'batting',
       'battingpost', 'collegeplaying', 'divisions', 'fielding',
       'fieldingof', 'fieldingofsplit', 'fieldingpost', 'halloffame',
       'homegames', 'leagues', 'managers', 'managershalf', 'parks',
       'people', 'pitching', 'pitchingpost', 'salaries', 'schools',
       'seriespost', 'teams', 'teamsfranchises', 'teamshalf'],
      dtype=object)

## Question 1 answer

In [13]:
query="""
SELECT * FROM salaries;
"""

In [14]:
salaries = pd.read_sql_query(query, conn)
salaries.head()

,ID,yearID,teamID,team_ID,lgID,playerID,salary
0,1,1985,ATL,1918,NL,barkele01,870000.0
1,2,1985,ATL,1918,NL,bedrost01,550000.0
2,3,1985,ATL,1918,NL,benedbr01,545000.0
3,4,1985,ATL,1918,NL,campri01,633333.0
4,5,1985,ATL,1918,NL,ceronri01,625000.0


In [24]:
query="""
SELECT teamID, yearID, team_ID, SUM(salary) AS total_salary FROM salaries GROUP BY teamID, team_ID, yearID ORDER BY teamID, team_ID;
"""
yearly_salary = pd.read_sql_query(query, conn)
yearly_salary

,teamID,yearID,team_ID,total_salary
0,ANA,1997,2238,31135472.0
1,ANA,1998,2266,41281000.0
2,ANA,1999,2296,55388166.0
3,ANA,2000,2326,51464167.0
4,ANA,2001,2356,47535167.0
...,...,...,...,...
913,WAS,2012,2715,80855143.0
914,WAS,2013,2745,113703270.0
915,WAS,2014,2775,131983680.0
916,WAS,2015,2805,155587472.0


## Question 2 answer

In [29]:
query="""
SELECT * FROM fielding;
"""
players = pd.read_sql_query(query, conn)
players.head()

,ID,playerID,yearID,stint,teamID,team_ID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
0,1,abercda01,1871,1,TRO,8,NA,SS,1,1.0,24.0,1,3,2.0,0,NaN,NaN,NaN,NaN,NaN
1,2,addybo01,1871,1,RC1,7,NA,2B,22,22.0,606.0,67,72,42.0,5,NaN,NaN,NaN,NaN,NaN
2,3,addybo01,1871,1,RC1,7,NA,SS,3,3.0,96.0,8,14,7.0,0,NaN,NaN,NaN,NaN,NaN
3,4,allisar01,1871,1,CL1,3,NA,2B,2,0.0,18.0,1,4,0.0,0,NaN,NaN,NaN,NaN,NaN
4,5,allisar01,1871,1,CL1,3,NA,OF,29,29.0,729.0,51,3,7.0,1,NaN,NaN,NaN,NaN,NaN


In [28]:
query="""
SELECT ID, playerID,
MIN(yearID) AS first_year, Max(yearID) AS last_year FROM fielding
GROUP BY playerID;
"""
players2 = pd.read_sql_query(query, conn)
players2

,ID,playerID,first_year,last_year
0,132951,aardsda01,2004,2015
1,73018,aaronha01,1954,1976
2,67100,aaronto01,1962,1971
3,91124,aasedo01,1977,1990
4,111673,abadan01,2001,2003
...,...,...,...,...
19486,57773,zupofr01,1957,1961
19487,93989,zuvelpa01,1982,1991
19488,56124,zuverge01,1951,1959
19489,26721,zwilldu01,1910,1916


## Question 3 answer

In [30]:
query="""
SELECT * FROM allstarfull;
"""
allstar = pd.read_sql_query(query, conn)
allstar.head()

,ID,playerID,yearID,gameNum,gameID,teamID,team_ID,lgID,GP,startingPos
0,1,gomezle01,1933,0,ALS193307060,NYA,921,AL,1,1.0
1,2,ferreri01,1933,0,ALS193307060,BOS,912,AL,1,2.0
2,3,gehrilo01,1933,0,ALS193307060,NYA,921,AL,1,3.0
3,4,gehrich01,1933,0,ALS193307060,DET,919,AL,1,4.0
4,5,dykesji01,1933,0,ALS193307060,CHA,915,AL,1,5.0


In [38]:
query="""
SELECT *, SUM(gameNum) AS game_count FROM allstarfull
GROUP BY playerID ORDER by game_count DESC;
"""
allstar_players = pd.read_sql_query(query, conn)
allstar_players.head()

,ID,playerID,yearID,gameNum,gameID,teamID,team_ID,lgID,GP,startingPos,game_count
0,1130,aaronha01,1955,0,NLS195507120,ML1,1273,NL,1,NaN,9
1,741,berrayo01,1948,0,ALS194807130,NYA,1161,AL,0,NaN,9
2,1177,boyerke01,1956,0,ALS195607100,SLN,1294,NL,1,5.0,9
3,1081,burgesm01,1954,0,ALS195407130,PHI,1260,NL,1,NaN,9
4,1361,cepedor01,1959,1,NLS195907070,SFN,1341,NL,1,3.0,9


In [43]:
query="""
SELECT playerID, team_ID,game_count FROM
(SELECT *, SUM(gameNum) AS game_count FROM allstarfull
GROUP BY playerID )
WHERE game_count == 9;
"""
best_allstar_players = pd.read_sql_query(query, conn)
best_allstar_players

,playerID,team_ID,game_count
0,aaronha01,1273,9
1,berrayo01,1161,9
2,boyerke01,1294,9
3,burgesm01,1260,9
4,cepedor01,1341,9
5,facero01,1340,9
6,foxne01,1203,9
7,kalinal01,1271,9
8,mantlmi01,1225,9
9,matheed01,1239,9


## Question 4 answer

In [44]:
query="""
SELECT * FROM CollegePlaying;
"""
colleges = pd.read_sql_query(query, conn)
colleges.head()

,ID,playerID,schoolID,yearID
0,1,aardsda01,pennst,2001
1,2,aardsda01,rice,2002
2,3,aardsda01,rice,2003
3,4,abadan01,gamiddl,1992
4,5,abadan01,gamiddl,1993


In [48]:
query="""
SELECT schoolID, COUNT(DISTINCT playerID) as player_count FROM CollegePlaying GROUP BY schoolID ORDER BY player_count DESC;
"""
most_productive_colleges = pd.read_sql_query(query, conn)
most_productive_colleges.head(1)

,schoolID,player_count
0,texas,107
